# 2.1 線形最適化問題入門

## 2.1節の例題を解く(二次元の例)

In [1]:
from pulp import *

In [2]:
# 問題オブジェクトを生成
prob = LpProblem(name='LP-sample', sense=LpMaximize)
# 変数を定義
x1 = LpVariable('x1', lowBound=0.0)
x2 = LpVariable('x2', lowBound=0.0)

# 目的関数と制約条件
prob += 2*x1 + 3*x2 # 目的関数
prob += x1 + 3*x2 <= 9, 'ineq1' # 制約に名前をつけることができる(option)
prob += x1 + x2 <= 4, 'ineq2'
prob += x1 + x2 <= 10, 'ineq3'
print(prob)

# 最適化ソルバーで最適化計算
prob.solve()

# 結果の確認
print(LpStatus[prob.status]) # status確認
print('Optimal Value = ', value(prob.objective))
for v in prob.variables():
    print(v.name, '=', value(v))


LP-sample:
MAXIMIZE
2*x1 + 3*x2 + 0
SUBJECT TO
ineq1: x1 + 3 x2 <= 9

ineq2: x1 + x2 <= 4

ineq3: x1 + x2 <= 10

VARIABLES
x1 Continuous
x2 Continuous

Optimal
Optimal Value =  10.5
x1 = 1.5
x2 = 2.5


## 生産計画問題

In [3]:
import numpy as np

In [4]:
A = np.array([[3,1,2], [1,3,0], [0,2,4]])
c = np.array([150, 200, 300])
b = np.array([60, 36, 48])
(m,n) = A.shape

In [5]:
# 問題オブジェクトを生成
prob = LpProblem(name='Production', sense=LpMaximize)
# 変数を定義
x = [LpVariable('x'+str(i+1), lowBound=0) for i in range(n)]

# 目的関数と制約条件
prob += lpDot(c, x) # 目的関数
for i in range(m):
    prob += lpDot(A[i], x) <= b[i], 'ineq'+str(i)
print(prob)

# 最適化ソルバーで最適化計算
prob.solve()

# 結果の確認
print(LpStatus[prob.status]) # status確認
print('Optimal Value = ', value(prob.objective))
for v in prob.variables():
    print(v.name, '=', value(v))

Production:
MAXIMIZE
150*x1 + 200*x2 + 300*x3 + 0
SUBJECT TO
ineq0: 3 x1 + x2 + 2 x3 <= 60

ineq1: x1 + 3 x2 <= 36

ineq2: 2 x2 + 4 x3 <= 48

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous

Optimal
Optimal Value =  5800.0
x1 = 12.0
x2 = 8.0
x3 = 8.0


# 双対性
- 計算結果の最適解が本当に信頼できるのか？という観点で双対性を考える

In [6]:
X = np.array([v.varValue for v in prob.variables()]) # 得られた解を列挙したベクトルを作っている
np.all(np.abs(b - np.dot(A, X)) <= 1.0e-5)
# AとXの内積をとり、bと比較してAX=bが満たされるのかを確認

True

# 双対定理

In [15]:
# 上記の双対問題を解く
AT = A.T

# 双対問題のオブジェクトを生成
dual = LpProblem(name='Dual_Production', sense=LpMinimize)
# 変数を定義
y = [LpVariable('y'+str(i+1), lowBound=0) for i in range(m)]

# 目的関数と制約条件
dual += lpDot(b, y) # 目的関数
for j in range(n):
    dual += lpDot(AT[j], y) >= c[j], 'ineq'+str(j)
print(dual)

# 最適化ソルバーで最適化計算
dual.solve()

# 結果の確認
print(LpStatus[dual.status]) # status確認
print('Optimal Value of dual problem = ', value(dual.objective))
for v in dual.variables():
    print(v.name, '=', v.varValue)

Dual_Production:
MINIMIZE
60*y1 + 36*y2 + 48*y3 + 0
SUBJECT TO
ineq0: 3 y1 + y2 >= 150

ineq1: y1 + 3 y2 + 2 y3 >= 200

ineq2: 2 y1 + 4 y3 >= 300

VARIABLES
y1 Continuous
y2 Continuous
y3 Continuous

Optimal
Optimal Value of dual problem =  5799.999996
y1 = 44.444444
y2 = 16.666667
y3 = 52.777778


In [16]:
Y = np.array([v.varValue for v in dual.variables()])
np.all(np.abs(np.dot(AT, Y) - c) <= 1.0e-5)

True